# Analyse d'accélération
## Comparer la vitesse CPU vs GPU
Avant d'utiliser massivement les GPUs d'une grappe de calcul, il faut tout d'abord que l'application ou l'algorithme puisse démontrer une "bonne performance" en utilisant plusieurs processeurs en parallèle.

Quelques définitions :
* **Temps écoulé** = temps d'exécution total que l'on perçoit et non le temps CPU
* **Accélération** = (temps avec un processeur) / (temps avec parallélisme)
* **Efficacité** = (Accélération) / (nombre de processeurs)

Le coût d'un noeud GPU étant cinq fois supérieur à celui d'un noeud
régulier, l'utilisation d'un seul GPU doit permettre une accélération
d'au moins cinq fois (5x) la vitesse de huit (8) à douze (12) processeurs.
* **Accélération** = (temps avec 8 à 12 processeurs) / (temps avec un accélérateur)

#### Exercice - Calcul d'accélération et d'efficacité
En supposant les résultats suivants pour un programme parallèle lancé sur $n$ processeurs :

| $n$ proc. |temp (s)|
|:---------:|:------:|
|     1     |  9.876 |
|     4     |  5.220 |
| 8 (1 gpu) |  0.035 |

Étant donné le script [`scripts/calc-acc-eff.sh`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/calc-acc-eff.sh) :
```Bash
bash scripts/calc-acc-eff.sh 1:9.876 4:5.220
bash scripts/calc-acc-eff.sh 1:9.876 8:0.035
```

## Extrapoler les ressources nécessaires
### Efficacité cible du calcul parallèle
* Une efficacité de 80%, voire 90%, devrait être un seuil minimal pour les tâches parallèles. Or, il existe un **nombre maximal de processeurs** à utiliser pour respecter ce seuil :
  * Voir la figure dans la page [loi d'Amdahl](https://fr.wikipedia.org/wiki/Loi_d%27Amdahl)
  * Voir aussi la section suivante
* On vise aussi une consommation en mémoire-vive de l'ordre de 80% de ce qui est demandé à l'ordonnanceur Slurm

Rappel - vous pouvez obtenir ces pourcentages via les commandes `sacct -X` (surtout pour obtenir les numéros de tâches) et `seff`. Les valeurs à considérer sont :
* `CPU Utilized` et `CPU Efficiency`
* `Memory Utilized` et `Memory Efficiency`

## Loi d'Amdahl
Voici quelques définitions :
* $T_s$: temps requis pour une exécution avec un seul processeur (donc 100% séquentiel)
* $P$ : fraction de $T_s$ correspondant à des opérations **parallèles**, donc **divisible** par $n$ processeurs.
* $S$ : fraction de $T_s$ correspondant à des opérations **séquentielles**, donc **non-divisible** par $n$ processeurs.
 * Exemples d'opérations séquentielles : lecture-écriture d'un fichier, communications, synchronisation, etc.
* Dans ce modèle, $P + S = 1$, donc $S = 1 - P$

$$T_p(n) = T_s * \left(S + \frac{P}{n}\right) = T_s * \left(1 - P + \frac{P}{n}\right)$$

De là, on peut redéfinir l'accélération $A(n)$ selon $n$ processeurs et isoler $P$ pour éventuellement le calculer :

$$A(n) = \frac{T_p(1)}{T_p(n)} = \frac{T_s * \left(1 - P + \frac{P}{1}\right)}{T_s * \left(1 - P + \frac{P}{n}\right)} = \frac{1 - P + P}{1 - P + \frac{P}{n}} = \frac{1}{1 - P + \frac{P}{n}}$$

$$\frac{1}{A(n)} = 1 - P + \frac{P}{n} = 1 - P * \left(1 - \frac{1}{n}\right) \implies P * \left(1 - \frac{1}{n}\right) = 1 - \frac{1}{A(n)}$$

$$P = \frac{1 - \frac{1}{A(n)}}{1 - \frac{1}{n}}$$

Pour finalement imposer une efficacité $E(n)$ minimale $e$ afin de calculer le $n$ maximal :

$$E(n) = \frac{A(n)}{n} \geq e \implies A(n) \geq e * n \implies \frac{1}{1 - P + \frac{P}{n}} \geq e * n$$

$$1 \geq e * n * \left(1 - P + \frac{P}{n} \right) = e * (1 - P) * n + e * P$$

$$1 - e * P \geq (e - e * P) * n \implies \frac{1 - e * P}{e - e * P} \geq n$$

$$n \leq \frac{\frac{1}{e} - P}{1 - P}$$

De là, c'est possible de calculer $n$ lorsque $e = 0.8$, par exemple.

## Exercice - Taille maximale d'une tâche parallèle
Explorer le script [`scripts/extra/calc-n-max.sh`](https://github.com/calculquebec/utilisation-serveurs-calcul/blob/main/scripts/extra/calc-n-max.sh)
```Bash
cat scripts/extra/calc-n-max.sh
```

**Important** - ce script est conçu pour recevoir le résultat redirigé du script
[`scripts/calc-acc-eff.sh`](https://github.com/calculquebec/utilisation-serveurs-calcul/blob/main/scripts/calc-acc-eff.sh) dans son canal STDIN
* En cas de blocage du terminal, faites Ctrl+C

Pour calculer le `n_max` pour une efficacité de 80% :
```Bash
bash scripts/calc-acc-eff.sh 1:61.8 4:16.8 | bash scripts/extra/calc-n-max.sh
```

On note que le calcul de `n_max` peut donner une valeur qu'il faut arrondir à la baisse, car l'efficacité diminue en fonction du nombre de processeurs :
```
Metrique,Valeur
P,0.970874
n_max,9.58333
n_max_entier,9
```